In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.constants import G
from scipy.integrate import solve_ivp
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
# Define the constants and provide their units.

G = G.to(u.cm**3/(u.g * u.s**2)) # Gravitational constant

M_moon = 7.349e25 * u.g # Mass of Moon
M_earth = 5.97e27 * u.g # Mass of Earth
M_sun = 1.98e33 * u.g # Mass of Sun

R_earth = 6371 * u.km # Radius of Earth
R_moon = 1740 * u.km # Radius of Moon
# data from: https://science.nasa.gov/moon/facts/#:~:text=The%20Moon%20makes%20a%20complete,orbit%20us%20every%2029%20days.

k2 = 0.298 # Dimensionless number
Q_moon = 11.5 # Dimensionless tidal quality factor

a_moon0 = 384000 * u.km # Semimajoraxise distance of the Moon
a_earth = 1.49e8 * u.km # Semimajoraxise distance of Earth

I = 0.3299 * M_earth * R_earth ** 2 # Earth's moment of inertial

lod = 86164 * u.s # length of the sidereal day
Omega_earth = (2 * np.pi)/lod # Earth's angular velocity 

Ratio_sun_moon = 1/4.7 # Ratio between Solar tidal torque and Lunar tidal torque

#### Now we can print out those constants with their units shown:

In [ ]:
G

In [ ]:
M_moon

In [ ]:
M_earth

In [ ]:
M_sun

In [ ]:
R_earth

In [ ]:
k2

In [ ]:
Q_moon

In [ ]:
a_moon0

In [ ]:
a_earth

In [ ]:
I

In [ ]:
lod

In [ ]:
Omega_earth

In [ ]:
Ratio_sun_moon

Notice there are some constans not using cgs unit.

### Task 1: Convert the constants to CGS units.

In [ ]:
R_earth = R_earth.to(u.cm) # convert Earth radius into cm
R_earth

In [ ]:
R_moon = R_moon.to(u.cm) # convert Moon radius into cm
R_moon

In [ ]:
a_moon0 = a_moon0.to(u.cm) # convert Semimajoraxise distance of the Moon into cm
a_moon0

In [ ]:
a_earth = a_earth.to(u.cm) # convert Semimajoraxise distance of Earth into cm
a_earth

In [ ]:
I = I.to(u.g * (u.cm**2)) # convert Earth's moment of inertial into g/cm^2
I

#### Task 1.2: Calculate L_earth, S_earth and L_moon:

In [ ]:
L_earth0 = M_earth * np.sqrt(G*(M_sun + M_earth) * a_earth)
L_earth0 # Initial value of Earth's angular momentum: this is a constant.

In [ ]:
S_earth0 = I * Omega_earth
S_earth0 # Initial value of Earth's spin angular momentum.

In [ ]:
def L_moon(a_moon):
    """
    Calculate the angular momentum of the Moon.

    This function computes the angular momentum of the Moon given its semimajor axis distance. 
    The result is returned as an astropy.units.quantity.Quantity object with units of cm^2 * g * s^-1.

    Parameters:
    
    a_moon : astropy.units.quantity.Quantity
    
        Semimajor axis distance of the Moon in centimeters.

    Returns:
    
    L_moon : astropy.units.quantity.Quantity
    
        The angular momentum of the Moon in units of cm^2 * g * s^-1.
    """
    L_moon = M_moon * np.sqrt(G*(M_earth + M_moon) * a_moon)
    return L_moon

In [ ]:
L_moon0 = L_moon(a_moon0) # Lunar angular momentum (L_moon) will change as the semi-major axis of the Moon (a_moon) varies.
L_moon0 # Initial(today's) value of the Moon's angular momentum

### Task 2:

In [ ]:
def T_moon(a_moon):
    """
    Calculate the tidal torque on the Moon.

    This function computes the tidal torque exerted on the Moon given its semimajor axis distance. 
    The result is returned as an astropy.units.quantity.Quantity object with units of cm^3 * g * s^-2.

    Parameters:
    
    a_moon : astropy.units.quantity.Quantity
    
        Semimajor axis distance of the Moon in centimeters.

    Returns:
    
    T_moon : astropy.units.quantity.Quantity
    
        The tidal torque on the Moon in units of cm^3 * g * s^-2.
    """

    T_moon = 1.5 * ((G * M_moon ** 2 )/ a_moon) * (R_earth / a_moon) ** 5 *(k2 / Q_moon)   
    
    return T_moon

In [ ]:
T_moon0 = T_moon(a_moon0) # initial (today's) value of the Moon's tidal torque 
T_moon0

In [ ]:
T_sun = 1.5 * G * M_moon ** 2 * R_earth ** 5 * (k2 / Q_moon) * (1/4.7) * (1/a_moon0)** 6
# T_sun is a constant value since a_moon^6 cancel out

In [ ]:
T_sun0 = T_sun # initial (today's) value of the Solar's tidal torque 
T_sun0

In [ ]:
# We can see that our calculation result is consistent with the given ratio
T_sun0/T_moon0

In [ ]:
Ratio_sun_moon

### Task 3: Calculate the three timescales associated with equations (1) to (3).

In [ ]:
t_1 = L_earth0 / T_sun0 # this is a constant value.
t_1_yr = t_1.to(u.yr)
t_1_yr

In [ ]:
t_2 = S_earth0 / (- T_sun0 - T_moon0)
t_2_yr = t_1.to(u.yr)
t_1_yr

In [ ]:
t_3 = L_moon0 / T_moon0
t_3_yr = t_1.to(u.yr)
t_3_yr

### Task 4: Define a function that evaluates the right-hand side (RHS) of equations (1) to (3):

In [ ]:
def ax(lm):
    """
    Calculate the semimajor axis distance of the Moon.

    This function computes the semimajor axis distance of the Moon given its angular momentum. 
    The result is returned as an astropy.units.quantity.Quantity object with units of centimeters.

    Parameters:
    
    lm : astropy.units.quantity.Quantity
    
        Angular momentum of the Moon in cm^2 * g * s^-1.

    Returns:
    
    a_moon : astropy.units.quantity.Quantity
    
        The semimajor axis distance of the Moon in units of cm.
    """
    
    a_moon = (lm / M_moon) ** 2 / (G * (M_earth + M_moon))
    
    return a_moon

In [ ]:
def rhs(t, W):
    """
    Calculate the right-hand side of equations (1) to (3)
    
    l : Earth's angular momentum
    s : Earth spin angular momentum
    lm : Moon's angular momentum
    
    This function defines the right-hand side of equations (1) to (3), given time and a parameter vector W, which contains l, s, and lm.
    The equations represent the rate of change of l, s, and lm.
    
    Parameters:
    
    t : tuple
    
        Time in seconds.
    
    W : list
    
        Parameter vector containing the values of l, s, and lm.

    Returns:
    
    list
    
        The list of derivatives of l, s, and lm with respect to time.
    """

def rhs(t, W):
    l, s, lm = W
    
    axx = ax(lm)
    
    ts = T_sun.value # We only want to take the value without unit.
    tm = T_moon(axx).value # We only want to take the value without unit.
    
    l_dot = ts 
    s_dot = - ts - tm
    lm_dot = tm
    
    return [l_dot, s_dot, lm_dot]

### Task 5: Using solve_ivp to solve the right-hand side (RHS) and find the time it takes for the Moon to form, in years.

In [ ]:
# Ser initial conditions 
l0 = L_earth0.value
s0 = S_earth0.value
lm0 = L_moon0.value

W0 = [l0, s0, lm0] # list without units

# Time span unit in seacond
t_span = (0, -1e30)

N = 31536000 # 1yr = 365 days * 24hr/day * 60 mins/hr * 60 s/min = 31536000s
t_eval = np.linspace(*t_span, N)

# Evaluates the right-hand side (RHS) of equations (1) to (3):
sol = solve_ivp(rhs, t_span, W0)

In [ ]:
# print out result 
sol

### Task 6: Create a figure displaying the graph of a_moon versus time (t)."

In [ ]:
# Extract the results and assign units.
t = sol.t * u.s
l = sol.y[0] * ((u.cm**2 * u.g)/u.s)
s = sol.y[1] * ((u.cm**2 * u.g)/u.s)
lm = sol.y[2] * ((u.cm**2 * u.g)/u.s)

In [ ]:
age = t.to(u.Gyr) # convert unit into billions of years from seconds
age

In [ ]:
a_moon_km = ax(lm).to(u.km) # convert unit into km from cm 
a_moon_km

In [ ]:
plt.plot(age, a_moon_km, color='r', label='a_moon(t)')
plt.title("Evolution of the Moon's Semimajor Axis with Age")
plt.xlabel('Age(Gyr)')
plt.ylabel('Simemajor axis (km)')
plt.savefig("age.pdf") # save as a pdf file
plt.grid(True)
plt.legend()
plt.show()

### Task 7: Make a figure showing the length of the day (in hours) versus age, with the same note as in the previous question.

In [ ]:
def find_lod(s):
    """
    Calculate the length of the day with different Earth's spin angular momentum.

    This function computes the length of the day given Earth's spin angular momentum.
    
    Parameters:
    
    s : astropy.units.quantity.Quantity
    
        Earth's spin angular momentum.

    Returns:
    
    lod : astropy.units.quantity.Quantity
    
        The length of the day in seconds.
    """
    Omega_earth  = s / I # Earth's angular velocity
    
    lod = (2 * np.pi)/Omega_earth  
    
    return lod

In [ ]:
lod_hr = find_lod(s).to(u.hr) # convert unit into hours from second
lod_hr

In [ ]:
plt.plot(age, lod_hr, color='g', label='Length 0f Day')
plt.title("Temporal Evolution of Earth's Day Length")
plt.xlabel('Age (Gyr)')
plt.ylabel('Length of Day (hours)')
plt.savefig("lod.pdf") # save as a pdf file
plt.grid(True)
plt.legend()
plt.show()

### Task 8: What is the Roche radius. Assuming that the moon did form there, what was the length of the day at the time of the Moon’s formation? 

In [ ]:
rho_earth = M_earth /((4/3) * np.pi * R_earth ** 3) # Earth's density 
rho_earth

In [ ]:
rho_moon = M_moon /((4/3) * np.pi * R_moon ** 3) # Moon's density
rho_moon

In [ ]:
d = R_earth*((2*(rho_earth/rho_moon))** (1/3)) # calculate Roche radius
d_km = d.to(u.km) # convert Roche radius into km
d_km

In [ ]:
# Assuming that the moon did form there then a_moon = d
lmr  = np.sqrt((G * (M_earth + M_moon)) * d) * M_moon  # Moon's Angular momentum at Roche radius.

# We know that the Earth's spin angular momentum plus the Moon's angular momentum is considered constant. 
L_EM = S_earth0 + L_moon0 

# Therefore, using this constant, we can determine the Earth's spin angular momentum given the Moon's angular momentum at its Roche radius.
s_d = L_EM - lmr

In [ ]:
find_lod(s_d)

### Task 9: What's the age of the Moon, and of the Earth.

In [ ]:
Age_of_moon = 4.35 * u.Gyr
Age_of_Earth = 4.54 * u.Gyr


""" 

1. U.S. Geological Survey 1997, Age of the Earth, \url{https://pubs.usgs.gov/gip/geotime/age.html}

2.Dalrymple, G. Brent 2001, "The age of the Earth in the twentieth century: a problem (mostly) solved," \
Special Publications, Geological Society of London, 190, 205-221, \url{https://doi.org/10.1144/GSL.SP.2001.190.01.14}

3.National Aeronautics and Space Administration (NASA), Moon Fact Sheet,
\url{https://nssdc.gsfc.nasa.gov/planetary/factsheet/moonfact.html}

4.National Aeronautics and Space Administration (NASA), Earth Fact Sheet, 
\url{https://nssdc.gsfc.nasa.gov/planetary/factsheet/earthfact.html}

5.Borg, L. E., Gaffney, A. M., & Shearer, C. K. 2015, "A review of lunar chronology revealing a preponderance of 4.34–4.37 Ga ages,"\
Meteoritics & Planetary Science, 50, 715-732, \url{https://doi.org/10.1111/maps.12371}

"""

In [ ]:
print(f'Age of the Moon is: {Age_of_moon}; Age of the Earth is: {Age_of_Earth}')

### Save all the output result in one signal pdf file.

In [ ]:
with PdfPages('result_graph.pdf') as pdf:
 
# save all the figure in one pdf file named output.

    plt.plot(age, a_moon_km, color='r', label='a_moon(t)')
    plt.title("Evolution of the Moon's Semimajor Axis with Age")
    plt.xlabel('Age(Gyr)')
    plt.ylabel('Simemajor axis (km)')
    plt.grid(True)
    plt.legend()
    pdf.savefig()
   



    plt.plot(age, lod_hr, color='g', label='Length 0f Day')
    plt.title("Temporal Evolution of Earth's Day Length")
    plt.xlabel('Age (Gyr)')
    plt.ylabel('Length of Day (hours)')
    plt.grid(True)
    plt.legend()
    pdf.savefig() 
  
